![Modelo ER](Modelo_Entidad_Relacion.png)

# 🛠️ Creación de Base de Datos: Proveedores y Piezas


Este notebook crea una base de datos relacional sobre proveedores, piezas, categorías y suministros. Incluye:
- Creación de tablas con claves primarias y foráneas
- Inserción de datos representativos
- Consultas de prueba para verificar la integridad

Base de datos: `proveedores.db`


In [ ]:

import sqlite3
import pandas as pd

# Crear conexión
conn = sqlite3.connect("proveedores.db")
cursor = conn.cursor()


## 📐 Crear tablas

In [ ]:

cursor.executescript("""
DROP TABLE IF EXISTS suministro;
DROP TABLE IF EXISTS pieza;
DROP TABLE IF EXISTS proveedor;
DROP TABLE IF EXISTS categoria;

CREATE TABLE proveedor (
    codigo_proveedor INTEGER PRIMARY KEY,
    nombre TEXT NOT NULL,
    direccion TEXT,
    ciudad TEXT,
    provincia TEXT
);

CREATE TABLE categoria (
    codigo_categoria INTEGER PRIMARY KEY,
    nombre TEXT NOT NULL
);

CREATE TABLE pieza (
    codigo_pieza INTEGER PRIMARY KEY,
    nombre TEXT NOT NULL,
    color TEXT,
    precio REAL,
    codigo_categoria INTEGER,
    FOREIGN KEY (codigo_categoria) REFERENCES categoria(codigo_categoria)
);

CREATE TABLE suministro (
    codigo_proveedor INTEGER,
    codigo_pieza INTEGER,
    fecha TEXT,
    cantidad INTEGER,
    PRIMARY KEY (codigo_proveedor, codigo_pieza, fecha),
    FOREIGN KEY (codigo_proveedor) REFERENCES proveedor(codigo_proveedor),
    FOREIGN KEY (codigo_pieza) REFERENCES pieza(codigo_pieza)
);
""")


## 📦 Insertar datos de ejemplo

In [ ]:

# Categorías
categorias = [
    (1, "Electrónica"),
    (2, "Ferretería"),
    (3, "Mecánica")
]

cursor.executemany("INSERT INTO categoria VALUES (?, ?);", categorias)

# Proveedores
proveedores = [
    (100, "Suministros León", "Av. Roma 12", "León", "Castilla y León"),
    (101, "Distribuciones Norte", "C/ Gran Vía 5", "Oviedo", "Asturias")
]

cursor.executemany("INSERT INTO proveedor VALUES (?, ?, ?, ?, ?);", proveedores)

# Piezas
piezas = [
    (200, "Tornillo M5", "plata", 0.15, 2),
    (201, "Chip XYZ", "verde", 5.00, 1),
    (202, "Filtro aceite", "negro", 3.50, 3)
]

cursor.executemany("INSERT INTO pieza VALUES (?, ?, ?, ?, ?);", piezas)

# Suministros
suministros = [
    (100, 200, "2024-03-01", 500),
    (100, 201, "2024-03-02", 50),
    (101, 202, "2024-03-01", 20),
    (100, 200, "2024-04-01", 300)  # histórico diferente fecha
]

cursor.executemany("INSERT INTO suministro VALUES (?, ?, ?, ?);", suministros)

conn.commit()


## 🔍 Consultas de prueba

In [ ]:

# Ver todos los proveedores
pd.read_sql_query("SELECT * FROM proveedor", conn)


In [ ]:

# Ver suministros de un proveedor
pd.read_sql_query("""
SELECT p.nombre AS pieza, s.fecha, s.cantidad
FROM suministro s
JOIN pieza p ON p.codigo_pieza = s.codigo_pieza
WHERE s.codigo_proveedor = 100
""", conn)


✅ Base de datos creada y poblada con éxito.